In [2]:
! pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=2744893 sha256=e30b708984c4d0c109ab858c7829083f7259690b9056e083a8215bf8fe322cda
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


**Important:** you will have to re-run this cell after you you get disconnected for a fairly long time.

**Below, we write a Hello World for MPI**

In [3]:
%%writefile mpi_hello_world.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    print("Greetings from process {} of {} on {}"\
    .format(id, numProcesses, myHostName))

########## Run the main function
main()


Writing mpi_hello_world.py


In [5]:
# Write MPI hostfile
import socket
import os
hostname = socket.gethostname()
#cores = os.cpu_count() # Count the number of cores in a computer
#print("Using " + str(cores) + " cores")
cores = len(os.sched_getaffinity(0))
print("Using " + str(cores) + " cores")

mpi_hostfile = hostname + "   slots=" + str(cores)
f = open("mpi_hostfile", "w")
f.write(mpi_hostfile)
f.close()

Using 2 cores


In [4]:
! mpirun --hostfile mpi_hostfile --allow-run-as-root -np 2 python mpi_hello_world.py

Greetings from process 0 of 2 on 249f2184fd6a
Greetings from process 1 of 2 on 249f2184fd6a


**Now, we try a simple parallel program to calculate pi**

In [6]:
%%writefile calcPiMPI.py

import sys
import time

#####################
# MPI Setup
#####################
from mpi4py import MPI
comm = MPI.COMM_WORLD
root = 0  # specify the root or rank 0 process
rank = comm.Get_rank()  # this process' unique integer identifier
num_procs = comm.Get_size() # total number of processes
####################

# Distributed function to compute pi
def Pi(num_steps):
	step = 1.0/num_steps
	sum = 0
	for i in range(rank, num_steps, num_procs):
		x = (i + 0.5)*step
		sum = sum + 4.0/(1.0 + x*x)
	my_part_of_pi = step * sum

	pi = comm.reduce(my_part_of_pi, MPI.SUM, root)
	return pi

if len(sys.argv) != 2:
	print("Usage: ", sys.argv[0], "<number_of_steps>")
	sys.exit(1)

num_steps = int(sys.argv[1], 10)

start = time.perf_counter() #start timing
pi = Pi( num_steps)
end = time.perf_counter() #end timing

if rank == root:
	print("Pi = ", pi, ",  Time elapsed: ", end-start)
sys.exit(0)

Writing calcPiMPI.py


In [13]:
! mpirun --hostfile mpi_hostfile --allow-run-as-root -np 1 python calcPiMPI.py 10000000


Pi =  3.141592653589731 ,  Time elapsed:  1.6716909239999893
